In [158]:
import json
import csv
import logging
import itertools
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 
from pandas import read_csv
from functools import reduce
from collections import defaultdict

In [159]:
etl = 'cambodia2newaqm'

if etl == 'pototan2newaqm':
    resident_file = 'file/parsed_output/Pototan/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Pototan/output.csv'
    parsed_output_json_file = 'file/parsed_output/Pototan/output.json'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Cambodia/resident-3999.csv'
    parsed_output_csv_file = 'file/parsed_output/Cambodia/output.csv'
    parsed_output_json_file = 'file/parsed_output/Cambodia/output.json'
    
elif etl == 'isabela2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Isabela/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Isabela/output.csv'
    parsed_output_json_file = 'file/parsed_output/Isabela/output.json'

In [160]:
resident_df = pd.DataFrame()
resident_df = pd.read_csv(resident_file) 
#hc_df.reset_index(level=0, inplace=True)
resident_df.head(2)

,_id,_index_map,health_information.alcohol_within_week,health_information.times_exposed_to_t_smoke,personal_info.source_of_income,identity_consent.personal_id_no,identity_consent.town,dental_health.owning_toothbrush,identity_consent.identity_document,health_information.how_taking_prescribed_dose,...,personal_info.gender,pregnancy.is_first_pregnancy,health_information.has_health_insurance_plan,household.mostly_used_to_wash_hands,household.type_of_sanitation,household.no_of_people_household,dental_health.times_brush_teeth_daily,health_information.blood_pressure_1st_systole,end,health_information.diagnosed
0,173,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,F,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2018-10-02T10:08:13.645+07,NaN
1,176,NaN,NaN,NaN,NaN,NaN,_,NaN,BC,NaN,...,M,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2018-10-02T11:00:08.474+07,NaN


In [161]:
resident_df.T

,0,1,2,3,4,5,6,7,8,9,...,3949,3950,3951,3952,3953,3954,3955,3956,3957,3958
_id,173,176,178,181,179,172,174,175,177,180,...,4286,4287,4288,4289,4290,4291,4292,4293,4294,4295
_index_map,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health_information.alcohol_within_week,NaN,NaN,NaN,NaN,NaN,5,0,0,0,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health_information.times_exposed_to_t_smoke,NaN,NaN,NaN,NaN,NaN,53,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
personal_info.source_of_income,NaN,NaN,NaN,NaN,NaN,BUS,NOA,OTH,OTH,NaN,...,EMP,EMP,NaN,EMP,OTH,OTH,NaN,NaN,OTH,NaN
identity_consent.personal_id_no,NaN,NaN,l,NaN,NaN,1234,051517261,180146096 (01),180279323(01),NaN,...,NaN,NaN,NaN,180727256,NaN,NaN,NaN,NaN,NaN,NaN
identity_consent.town,_,_,_,_,_,_,_,_,_,_,...,_,_,_,_,_,_,_,_,_,_
dental_health.owning_toothbrush,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,N,Y,...,Y,Y,NaN,Y,Y,Y,NaN,NaN,N,N
identity_consent.identity_document,BC,BC,BC,BC,BC,BC,NID,NID,NID,BC,...,BC,BC,BC,NID,BC,BC,BC,BC,BC,BC
health_information.how_taking_prescribed_dose,NaN,NaN,NaN,NaN,NaN,PAF,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
def concat_values(items):
    _list = []
    _index_map = ''
    _id = ''
    
    for index ,value in items.iteritems():
        _new_item = ''
        
    
        if index == "_id":
            _id = value
            #print(_index_map)
            _list.append(_id)
           
        elif index == "_index_map":
            _index_map = value
            
            _list.append(_index_map)
            #TODO: MUST BE A FUNCTION --> Parser of index_map
        else:
            if len(index) > 1 :
                
                _new_index = index.split(sep='.')
                
                
                if pd.isna(_index_map) :
                    _new_index[0] = _new_index[0]
                
                else:
                    
                    _new_index[0] = _new_index[0] + '|' + str(_index_map)
                    
                _edited_index = '.'.join(_new_index)
                _edited_index

                    #===========
                    #_new_item = index + ':"' + str(value) + '"'
                _new_item = _edited_index + ":'" + str(value) + "'"
                _list.append(_new_item)
            else:
                continue
            
    return _list    

In [163]:
new_residents_df = pd.DataFrame()
new_resident_df = resident_df.T.apply(concat_values)
new_resident_df = new_resident_df.drop(index='_index_map')
#TODO: Change idnetifier _id formid id accordingly
#new_resident_df = new_resident_df.drop(index='_id')
new_resident_df 

,0,1,2,3,4,5,6,7,8,9,...,3949,3950,3951,3952,3953,3954,3955,3956,3957,3958
_id,173,176,178,181,179,172,174,175,177,180,...,4286,4287,4288,4289,4290,4291,4292,4293,4294,4295
health_information.alcohol_within_week,health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'5.0',health_information.alcohol_within_week:'0.0',health_information.alcohol_within_week:'0.0',health_information.alcohol_within_week:'0.0',health_information.alcohol_within_week:'nan',...,health_information.alcohol_within_week:'0.0',health_information.alcohol_within_week:'0.0',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan',health_information.alcohol_within_week:'nan'
health_information.times_exposed_to_t_smoke,health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'5...,health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',...,health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan',health_information.times_exposed_to_t_smoke:'nan'
personal_info.source_of_income,personal_info.source_of_income:'nan',personal_info.source_of_income:'nan',personal_info.source_of_income:'nan',personal_info.source_of_income:'nan',personal_info.source_of_income:'nan',personal_info.source_of_income:'BUS',personal_info.source_of_income:'NOA',personal_info.source_of_income:'OTH',personal_info.source_of_income:'OTH',personal_info.source_of_income:'nan',...,personal_info.source_of_income:'EMP',personal_info.source_of_income:'EMP',personal_info.source_of_income:'nan',personal_info.source_of_income:'EMP',personal_info.source_of_income:'OTH',personal_info.source_of_income:'OTH',personal_info.source_of_income:'nan',personal_info.source_of_income:'nan',personal_info.source_of_income:'OTH',personal_info.source_of_income:'nan'
identity_consent.personal_id_no,identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'l',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'1234',identity_consent.personal_id_no:'051517261',identity_consent.personal_id_no:'180146096 (01)',identity_consent.personal_id_no:'180279323(01)',identity_consent.personal_id_no:'nan',...,identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'180727256',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan',identity_consent.personal_id_no:'nan'
identity_consent.town,identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',identity_consent.town:'_',ide

In [164]:
def insert_concat(items):
    test_df = pd.DataFrame()
    _test_obj = []
    
    for index ,value in items.iteritems():
        _kv = {}
        
        #TODO: change _id here in formid, id or id
        if index == '_id':
        #if index == 'id':
            #print(index)
            _test_id = value
            continue
        else:
            _kv["id"] = _test_id
            _kv["value"] = value
            
        _test_obj.append(_kv)
    return _test_obj
    

In [165]:
vertical_resident_df = []
vertical_resident_df = new_resident_df.apply(insert_concat)

flat = list(itertools.chain.from_iterable(vertical_resident_df))
flat_df = pd.DataFrame.from_dict(flat)
flat_df.head(5)

,id,value
0,173,health_information.alcohol_within_week:'nan'
1,173,health_information.times_exposed_to_t_smoke:'nan'
2,173,personal_info.source_of_income:'nan'
3,173,identity_consent.personal_id_no:'nan'
4,173,identity_consent.town:'_'


In [166]:
#TODO: delete if exisitng
flat_df.to_csv(parsed_output_csv_file, encoding='utf-8', mode='w', index=False)

In [167]:
hc_df = pd.DataFrame()
hc_df = pd.read_csv(parsed_output_csv_file,header=None) 
hc_df.head(5)

/Users/ghost/src/virtual_env/jupyther3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1
0,id,value
1,173,health_information.alcohol_within_week:'nan'
2,173,health_information.times_exposed_to_t_smoke:'nan'
3,173,personal_info.source_of_income:'nan'
4,173,identity_consent.personal_id_no:'nan'


In [168]:
def setValue(value, field, it):
    if isinstance(value, dict) or isinstance(value, list) and field not in it: # prevent override
            it[field] = value
    if not isinstance(value, dict) and not isinstance(value, list):
            it[field] = value

def findField(ds, k, keyVal, it):
    it = ds[k]
    if isinstance(it, list): # if array then find correct index
        index = int(keyVal[1])
        if len(it) > index:
            it = it[index]
    return it

def setAttribute(profile, lastKeys, field, value):
    it = {}
    for key in lastKeys:
        keyVal = key.split('|')
        k = keyVal[0]
        if not it:
            it = findField(profile, k, keyVal, it)
        else:
            it = findField(it, k, keyVal, it)

    if isinstance(it, list):
        index = int(lastKeys[-1].split('|')[1]) # get index from ['contactnumber/0']
        if len(it) > index: # find element and element attribute to set value
            setValue(value, field, it[index])
        else: # list is empty so create an element
            for i in range(len(it), index + 1, 1):
                it.append({})
            it[index][field] = value
    else:
        setValue(value, field, it)

def defineField(profile, field):
    keyVal = field.split('|')
    if len(keyVal) > 1:
        key = keyVal[0]
        if key not in profile:
            profile[key] = []
    elif field not in profile:
        profile[field] = {}

profiles = []

def properValue(value):
    proper = None
    try:
        proper = eval(value)
    except:
        proper = value
    return proper

# app starts here


print('parser running...')
with open(parsed_output_csv_file, encoding="utf-8") as csv_file:
#with open(resident_file, encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    profile = {}
    lastId = None
    firstProfile = True
    for row in csv_reader: # read the csv line by line
        if line_count > 0: # don't read columns
            # print(row[1].rsplit(':', 1))
            fieldValueRaw = row[1]
            fieldValue = fieldValueRaw.split(":", 1)
            rawFields = [x.strip() for x in fieldValue[0].split('.')]
            rawFields[-1] = rawFields[-1] + ':' + fieldValue[1]
            # stop updating profile when id changes
            # add profile and build it afterwards
            if lastId != row[0]: # new id means add and build profile
                if not firstProfile:
                    profile = {} # reset profile and build it
                profile['id'] = row[0]
                profiles.append(profile) # this profile will be built below
                lastId = row[0]
                firstProfile = False

            lastKeys = []
            for field in rawFields: # build the profile up
                keyVal = field.split(':', 1) # field and value format, we separate it
                if len(keyVal) == 1: # ['answers'] if field doesn't contain value we build it
                    if len(lastKeys) == 0:
                        defineField(profile, field)
                    else:
                        _keyVal = field.split('|')
                        if len(_keyVal) > 1:
                            setAttribute(profile, lastKeys, _keyVal[0], [])
                        else:
                            setAttribute(profile, lastKeys, field, {})
                    lastKeys.append(field)
                else: # ['type', 'mobile']
                    if len(lastKeys) == 0:
                        profile[keyVal[0]] = properValue(keyVal[1])
                    else:
                        setAttribute(profile, lastKeys, keyVal[0], properValue(keyVal[1]))
                    lastKeys.append(keyVal[0])
        line_count += 1

#os.makedirs('output', exist_ok=True)
#file = open(parsed_output_json_file, 'w')
#file.write(json.dumps(profiles, indent=4, sort_keys=True))
#file.write(json.dumps(profiles))
#file.close()

with open(parsed_output_json_file, 'w', encoding="utf-8") as outfile:
    json.dump(profiles, outfile, indent=4, sort_keys=True, ensure_ascii=False)
    
print('parser completed...')

parser running...
parser completed...


In [191]:
json_df = pd.read_json( 'file/parsed_output/Isabela/test.json').T

json_df
    

,0,1,2,3,4,5,6,7,8,9,...,6191,6192,6193,6194,6195,6196,6197,6198,6199,6200
answers,"{'Middle_Name': 'nan', 'Add_ID1_Value': 'nan',...","{'Middle_Name': 'Laddran', 'Add_ID1_Value': 'n...","{'Middle_Name': 'nan', 'Add_ID1_Value': 'nan',...","{'Middle_Name': 'Canceran', 'Add_ID1_Value': '...","{'Middle_Name': 'Canceran', 'Add_ID1_Value': '...","{'Middle_Name': 'nan', 'Add_ID1_Value': 'nan',...","{'Middle_Name': 'Soriano', 'Add_ID1_Value': '0...","{'Middle_Name': 'Caminto', 'Add_ID1_Value': 'n...","{'Middle_Name': 'Alluigan', 'Add_ID1_Value': '...","{'Middle_Name': 'Laggui', 'Add_ID1_Value': 'na...",...,"{'Middle_Name': 'liaban', 'Add_ID1_Value': 'na...","{'Middle_Name': 'maramag', 'Add_ID1_Value': 'n...","{'Middle_Name': 'malana', 'Add_ID1_Value': 'na...","{'Middle_Name': 'taguba', 'Add_ID1_Value': 'na...","{'Middle_Name': 'madamba', 'Add_ID1_Value': 'n...","{'Middle_Name': 'asuncion', 'Add_ID1_Value': '...","{'Middle_Name': 'maramag', 'Add_ID1_Value': '2...","{'Middle_Name': 'canceran', 'Add_ID1_Value': '...","{'Middle_Name': 'gamboa', 'Add_ID1_Value': 'na...","{'Middle_Name': 'magusib', 'Add_ID1_Value': 'n..."
createdBy,aguzman@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,...,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,csoriano@gmail.com,lcorpuz@gmail.com,csoriano@gmail.com,ecurampos@gmail.com,lcorpuz@gmail.com,csoriano@gmail.com
dateCreated,08/16/2018 at 10:25:07 PM GMT+08:00,07/12/2018 at 10:54:14 PM GMT+08:00,07/05/2018 at 10:08:44 PM GMT+08:00,07/07/2018 at 01:33:19 PM GMT+08:00,07/02/2018 at 08:44:11 PM GMT+08:00,07/15/2018 at 10:59:05 AM GMT+08:00,06/28/2018 at 10:07:54 AM GMT+08:00,07/02/2018 at 12:11:25 PM GMT+08:00,07/12/2018 at 08:57:59 AM GMT+08:00,07/02/2018 at 10:45:17 AM GMT+08:00,...,07/03/2018 at 12:51:43 PM GMT+08:00,06/28/2018 at 09:53:45 AM GMT+08:00,07/03/2018 at 12:43:39 PM GMT+08:00,08/05/2018 at 09:28:38 AM GMT+08:00,08/05/2018 at 02:37:26 PM GMT+08:00,08/02/2018 at 05:55:23 AM GMT+00:00,08/02/2018 at 12:58:47 PM GMT+08:00,08/14/2018 at 06:42:42 AM GMT+01:00,08/02/2018 at 04:20:44 AM GMT+00:00,07/03/2018 at 01:29:18 PM GMT+08:00
formId,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,...,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR,afL9Uv25E6rfZ7Uwkd9maR
formName,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,...,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions,1 - AQM Version 1 - As-Is - AccountQuestions
id,64c2e7de-0cc8-4df5-a54f-02398652c080,2f920d39-319e-4bf7-90c8-6133c3337af7,426635b5-2095-46b3-bd70-a2717bddc0af,56a4b723-4fb1-455c-81c3-43e816693752,76a72543-1e22-423e-875b-535e85c192f5,9f621021-aad3-47d4-a48a-d8c14e212199,a05309c1-fb64-4228-a557-ba073464b7a4,c688111e-c3d4-4cc1-bd1f-9baba9a496fd,cc81d2c3-ce60-423a-a8ae-8764dfb63bf7,eb8d372a-116d-466d-bae1-7

In [188]:
for index, row in json_df.iterrows():
    print(row)

0    {'id': '64c2e7de-0cc8-4df5-a54f-02398652c080',...
Name: 0, dtype: object
0    {'id': '2f920d39-319e-4bf7-90c8-6133c3337af7',...
Name: 1, dtype: object
0    {'id': '426635b5-2095-46b3-bd70-a2717bddc0af',...
Name: 2, dtype: object
0    {'id': '56a4b723-4fb1-455c-81c3-43e816693752',...
Name: 3, dtype: object
0    {'id': '76a72543-1e22-423e-875b-535e85c192f5',...
Name: 4, dtype: object
0    {'id': '9f621021-aad3-47d4-a48a-d8c14e212199',...
Name: 5, dtype: object
0    {'id': 'a05309c1-fb64-4228-a557-ba073464b7a4',...
Name: 6, dtype: object
0    {'id': 'c688111e-c3d4-4cc1-bd1f-9baba9a496fd',...
Name: 7, dtype: object
0    {'id': 'cc81d2c3-ce60-423a-a8ae-8764dfb63bf7',...
Name: 8, dtype: object
0    {'id': 'eb8d372a-116d-466d-bae1-7cf26957da96',...
Name: 9, dtype: object
0    {'id': 'f69962d8-a679-4067-9d68-1c35b4264835',...
Name: 10, dtype: object
0    {'id': 'c3c3fce8-7352-4b2d-9940-ef5ec29c2dbc',...
Name: 11, dtype: object
0    {'id': 'ab0e892a-127e-4159-aa21-fd17805cacf6',...
Name: 1